In [1]:
import sqlite3
import numpy as np
import pandas as pd
import itertools
import time

st = time.time()

conn = sqlite3.connect("stock.db")
ds_ratio = pd.read_sql_query('select STOCK_CODE, STOCK_FACTOR, STOCK_YEAR, STOCK_DATA from VCSC_FINANCIAL_RATIO', conn)
ds_price = pd.read_sql_query('select STOCK_CODE, STOCK_DATE, STOCK_CLOSE from VCSC_STOCK_PRICE', conn)
conn.close()

print("1--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_weight = np.arange(0.0, 1.01, 0.01)
ds_weight = [x for x in itertools.permutations(ds_weight, 2) if sum(x) == 1]
ds_weight = pd.DataFrame(ds_weight,columns=('W1','W2'))

ds_ratio = ds_ratio.loc[ds_ratio['STOCK_FACTOR'].isin(['EPS (VND)', 'BVS'])]
ds_ratio = ds_ratio.loc[ds_ratio['STOCK_YEAR'].str.contains('TTM')==False]
ds_ratio['STOCK_DATA'] = ds_ratio['STOCK_DATA'].str.replace('-', '')
ds_ratio['STOCK_DATA'] = ds_ratio['STOCK_DATA'].str.replace(',', '')
ds_ratio['STOCK_DATA'] = pd.to_numeric(ds_ratio['STOCK_DATA'])
ds_ratio['STOCK_YEAR'] = pd.to_numeric(ds_ratio['STOCK_YEAR'])

ds_price['STOCK_DATE'] = pd.to_datetime(ds_price['STOCK_DATE'], format='%m/%d/%Y')
ds_price['STOCK_YEAR'] = ds_price['STOCK_DATE'].dt.year
ds_price['STOCK_QUARTER'] = ds_price['STOCK_YEAR'].astype('str') + ds_price['STOCK_DATE'].dt.quarter.astype('str')
ds_price['STOCK_CLOSE'] = ds_price['STOCK_CLOSE'].str.replace(',', '').astype(float)

print("2--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_price = ds_price.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'STOCK_QUARTER']).nth(0)
ds_price['NEXT-PRICE-Q'] = ds_price.groupby(level=0)['STOCK_CLOSE'].shift(-1)
ds_price = ds_price.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'STOCK_YEAR']).nth(0)
ds_price['NEXT-PRICE-Y'] = ds_price.groupby(level=0)['STOCK_CLOSE'].shift(-1)

ds_ratio = ds_ratio.merge(ds_price, left_on=['STOCK_CODE', 'STOCK_YEAR'], right_on=['STOCK_CODE', 'STOCK_YEAR'], how='right')
ds_ratio['XP'] = ds_ratio['STOCK_DATA'].div(ds_ratio['NEXT-PRICE-Q'])
ds_ratio['RETURN'] = ds_ratio['NEXT-PRICE-Y'].div(ds_ratio['NEXT-PRICE-Q'])
ds_ratio['RANK'] = ds_ratio.groupby(['STOCK_FACTOR', 'STOCK_YEAR'])['XP'].rank('dense', ascending=True)

print("3--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_p = ds_ratio.pivot_table(index=['STOCK_YEAR', 'STOCK_CODE'], columns='STOCK_FACTOR', values=['RANK', 'RETURN'])
ds_p.columns = [' '.join(col).strip() for col in ds_p.columns.values]
ds_p = ds_p.reset_index()

ds_p['TMP'] = 1
ds_weight['TMP'] = 1
ds_p = pd.merge(ds_p, ds_weight, on='TMP', how='outer')

ds_p['RANK'] = ds_p['RANK BVS'] * ds_p['W1'] + ds_p['RANK EPS (VND)'] * ds_p['W2']
ds_p['DECILE'] = (ds_p.groupby(['W1', 'W2', 'STOCK_YEAR'])['RANK']
                                .apply(lambda x: pd.qcut(x,10,duplicates='drop',labels=False)))
ds_p = ds_p.loc[ds_p['DECILE'] == 9.0]

print("4--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_p['RETURN'] = ds_p['RETURN BVS']
ds_p = ds_p.drop(columns=['STOCK_CODE', 'RANK BVS', 'RANK EPS (VND)', 'RETURN BVS', 'RETURN EPS (VND)'])
ds_p = ds_p.drop(columns=['TMP', 'RANK', 'DECILE'])

s6 = ds_p.groupby(['W1', 'W2', 'STOCK_YEAR']).agg({'RETURN':'mean'})
s6 = s6.reset_index()

print("5--- %s seconds ---" % (time.time() - st))
st = time.time()

s6.pivot_table(index=(['W1', 'W2']),columns='STOCK_YEAR', values='RETURN').sort_values(by=2017, ascending=False)


1--- 3.8083770275115967 seconds ---
2--- 5.671436309814453 seconds ---
3--- 1.2813634872436523 seconds ---
4--- 1.0759766101837158 seconds ---
5--- 0.04687690734863281 seconds ---


,STOCK_YEAR,2011,2012,2013,2014,2015,2016,2017
W1,W2,,,,,,,
0.72,0.28,1.270753,1.767061,1.619729,1.649356,1.171843,1.317622,0.956134
0.60,0.40,1.230526,1.724647,1.622871,1.556816,1.159047,1.293018,0.950020
0.73,0.27,1.270753,1.767061,1.626605,1.647700,1.171843,1.324726,0.948969
0.54,0.46,1.236963,1.722395,1.647560,1.564096,1.170184,1.265791,0.948773
0.53,0.47,1.204537,1.722395,1.647560,1.555092,1.164255,1.265791,0.948773
0.56,0.44,1.236963,1.722395,1.636774,1.564096,1.158284,1.272340,0.947513
0.57,0.43,1.236963,1.722395,1.636774,1.556816,1.170099,1.272340,0.947513
0.58,0.42,1.236963,1.722395,1.636774,1.556816,1.168194,1.281071,0.947513
0.59,0.41,1.230526,1.724647,1.622871,1.556816,1.159047,1.287820,0.946912
